# Retrieve Data from the  Wayback Machine

In [13]:
# Import main packages
import pandas as pd
import os
import gc

import requests
from datetime import datetime
from urllib.parse import urlparse
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import json

# Define directories
def find_directory_upwards(dir_name):
    """
    Find the absolute path to a directory with the given name by searching upwards
    from the current notebook's directory.

    Parameters:
    dir_name (str): The name of the directory to find.

    Returns:
    str: The absolute path to the directory if found, otherwise None.
    """
    # Get the current notebook's directory
    current_dir = os.getcwd()
    
    while True:
        # Check if the directory exists in the current directory
        potential_path = os.path.join(current_dir, dir_name)
        if os.path.isdir(potential_path):
            return potential_path
        
        # Move to the parent directory
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        
        # If the current directory is the root directory, stop searching
        if current_dir == parent_dir:
            break
        
        current_dir = parent_dir
    
    # If the directory is not found, return None
    return None

code_dir = find_directory_upwards('codes')
raw_data_dir = find_directory_upwards('raw_data')
int_data_dir = find_directory_upwards('int_data')
final_data_dir = find_directory_upwards('final_data')
output_dir = find_directory_upwards('output')

print(code_dir)
# print(raw_data_dir)
print(int_data_dir)
print(output_dir)

/home/sosajuanbautista/aeai-filestore/projects/agentic/codes
/home/sosajuanbautista/aeai-filestore/projects/agentic/int_data
/home/sosajuanbautista/aeai-filestore/projects/agentic/output


## Load URLs with price data

In [8]:
# Load URLs for top 20 commodity categories
path = os.path.join(final_data_dir, 'commodity_price_sources', 'df_top20_commodity_prices_search_results_evaluated.csv')
df = pd.read_csv(path)

# Add column base_url
df['base_url'] = df['url'].apply(lambda x: urlparse(x).netloc)

print(df.columns.values)
print(df.shape)

df.head()

['commodity' 'search_term' 'title' 'url' 'justification' 'raw_content'
 'base_url']
(161, 7)


,commodity,search_term,title,url,justification,raw_content,base_url
0,Rotary tiller or power tiller,Rotary tiller or power tiller blue book,Tarter Farm & Ranch 4 ft. Light-Duty Rotary Ti...,https://www.tractorsupply.com/tsc/product/farm...,The search result provides a specific product ...,Error\n \nSign In / Enroll\nGet rewarded!\nTar...,www.tractorsupply.com
1,Rotary tiller or power tiller,Rotary tiller or power tiller used prices,Rotary Tillers For Sale | MyLittleSalesman.com,https://www.mylittlesalesman.com/rotary-tiller...,The webpage provides a list of rotary tillers ...,Rotary Tillers For Sale | MyLittleSalesman.com...,www.mylittlesalesman.com
2,Rotary tiller or power tiller,Rotary tiller or power tiller price list,Tillers - Rototillers & Cultivators - The Home...,https://www.homedepot.com/b/Outdoors-Outdoor-P...,The webpage provides a list of various tiller ...,Select store\n...\n...\nCart\nSelect store\n.....,www.homedepot.com
3,Rotary tiller or power tiller,Power tiller price listings,Rotary Tillage Equipment For Sale in OKLAHOMA,https://www.tractorhouse.com/listings/rotary-t...,The search result provides a listing of rotary...,NaN,www.tractorhouse.com
4,Rotary tiller or power tiller,Power tiller price listings,Rotary Tillage Equipment For Sale | TractorHou...,https://www.tractorhouse.com/listings/for-sale...,The webpage provides a listing of rotary tilla...,NaN,www.tractorhouse.com


## Retrieve scraping timestamps
For each of URL with relevant price data, see how far back the Internet Archive has scraped it.

In [9]:
# Try scraping one sample URL
cdx_api = 'https://web.archive.org/cdx/search/cdx'

# Parameters for the API request
params = {
    'url': 'https://www.kbb.com/new-cars/',
    'output': 'json',
    'fl': 'timestamp',
    'filter': 'statuscode:200'
}
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(cdx_api, params=params, headers=headers, timeout=10)
data = response.json()
data

[['timestamp'],
 ['20090912133918'],
 ['20090913201302'],
 ['20090914043103'],
 ['20090914043108'],
 ['20090922170155'],
 ['20091019013245'],
 ['20091020081206'],
 ['20091028175907'],
 ['20100106114403'],
 ['20100119012758'],
 ['20100120032013'],
 ['20100203102530'],
 ['20100208124702'],
 ['20100209083218'],
 ['20100213054136'],
 ['20100218060420'],
 ['20100222082408'],
 ['20100228073200'],
 ['20100304161854'],
 ['20100304163913'],
 ['20100308153359'],
 ['20100313090619'],
 ['20100316215820'],
 ['20100327234823'],
 ['20100328021725'],
 ['20100401223705'],
 ['20100402123217'],
 ['20100407100209'],
 ['20100408081704'],
 ['20100408081704'],
 ['20100412025722'],
 ['20100412200458'],
 ['20100412200458'],
 ['20100418163537'],
 ['20100418163537'],
 ['20100419040711'],
 ['20100420042811'],
 ['20100427184452'],
 ['20100430202045'],
 ['20100504142222'],
 ['20100521234537'],
 ['20100528203459'],
 ['20100529100328'],
 ['20100531001332'],
 ['20100601052632'],
 ['20100602075702'],
 ['20100603223506'

In [10]:
def get_wm_timestamp_data(target_url, verbose=True):
    # Updated CDX API endpoint to use HTTPS
    cdx_api = 'https://web.archive.org/cdx/search/cdx'

    # Parameters for the API request
    params = {
        'url': target_url,
        'output': 'json',
        'fl': 'timestamp',
        'filter': 'statuscode:200'
    }

    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(cdx_api, params=params, headers=headers, timeout=10)

        if response.status_code != 200:
            if verbose:
                print(f"Error fetching data from CDX API for URL: {target_url}")
            return 'SATUS_CODE_ERROR'

        data = response.json()

        if len(data) < 2:
            if verbose:
                print(f"No data found for the URL: {target_url}")
            return 'NO_DATA'

        # Extract timestamps from the data
        timestamps = [entry[0] for entry in data[1:]]

        return timestamps

    except requests.exceptions.RequestException as e:
        if verbose:
            print(f"Request failed for URL {target_url}: {e}")
        return 'REQUEST_EXCEPTION'


In [ ]:
# RETRIEVE TIMESTAMPS FOR ORIGINAL/FULL URLs

# Create emtpy dictionary to store the archive dates
archive_dates_fullurl = {}

# Loop with progress bar using tqdm
for target_url in tqdm(list(set(df['url'].values)), desc="Processing URLs", unit="url"):

    # Call the function to get archive dates
    timestamps = get_wm_timestamp_data(target_url, verbose=False)

    # Store the archive dates in the dictionary
    archive_dates_fullurl[target_url] = timestamps

    # Sleep to avoid hitting the CDX API rate limit
    time.sleep(3)

# Save the dictionary as a JSON file
today = datetime.today().strftime('%Y-%m-%d_%H.%M')
path = os.path.join(final_data_dir, 'commodity_price_sources', 'wayback_machine', f'archive_dates_fullurls_{today}.json')
with open(path, 'w') as f:
    json.dump(archive_dates_fullurl, f)
    
archive_dates_fullurl

Processing URLs: 100%|██████████| 161/161 [11:27<00:00,  4.27s/url]


{'https://www.newegg.com/Desktop-Computer/Category/ID-228': ['20240926084111',
  '20240927094436',
  '20241001201009',
  '20241002162055',
  '20241003001244',
  '20241004005554',
  '20241004224850',
  '20241008014502',
  '20241009023803',
  '20241009190908'],
 'https://www.mylittlesalesman.com/rotary-tillers-for-sale-i6c766f0m0': 'NO_DATA',
 'https://www.gabion1.com/gabion_us_prices.htm': ['20141103214804',
  '20150204130703',
  '20150209021738',
  '20150210130105',
  '20150311050604',
  '20150405063340',
  '20150429135959',
  '20150530062933',
  '20150630032428',
  '20150801155916',
  '20150901053024',
  '20151001202844',
  '20170720151522',
  '20210415075216',
  '20211207211029'],
 'https://approvedcosts.com/copier/': ['20210621013143',
  '20210621013143',
  '20210919204158'],
 'https://www.staples.com/office-supplies/cat_SC1': ['20090804001135',
  '20090804041723',
  '20090806173239',
  '20090814051351',
  '20090822015556',
  '20090827091724',
  '20090901161706',
  '20090907092228',

In [16]:
# RETRIEVE TIMESTAMPS FOR BASE URLs

# Create emtpy dictionary to store the archive dates
archive_dates_baseurl = {}

# Loop with progress bar using tqdm
for target_url in tqdm(list(set(df['base_url'].values)), desc="Processing URLs", unit="url"):

    # Call the function to get archive dates
    timestamps = get_wm_timestamp_data(target_url, verbose=False)

    # Store the archive dates in the dictionary
    archive_dates_baseurl[target_url] = timestamps

    # Sleep to avoid hitting the CDX API rate limit
    time.sleep(3)

# Save the dictionary as a JSON file
today = datetime.today().strftime('%Y-%m-%d_%H.%M')
path = os.path.join(final_data_dir, 'commodity_price_sources', 'wayback_machine', f'archive_dates_baseurls_{today}.json')
with open(path, 'w') as f:
    json.dump(archive_dates_baseurl, f)

archive_dates_baseurl

Processing URLs: 100%|██████████| 75/75 [05:45<00:00,  4.61s/url]


{'apollotools.com': ['20030904040718',
  '20031012111010',
  '20031015120621',
  '20031123185542',
  '20031201033759',
  '20040129121556',
  '20040320184522',
  '20040320200652',
  '20040518030051',
  '20040525043011',
  '20040606015425',
  '20040608065840',
  '20040725184350',
  '20040726205234',
  '20040923135457',
  '20040923204621',
  '20041128071740',
  '20041202191624',
  '20050126173333',
  '20050203214129',
  '20050205060418',
  '20050211114013',
  '20050306105754',
  '20050406034146',
  '20050622234652',
  '20050818060318',
  '20051023014502',
  '20051024024111',
  '20051216041918',
  '20060118130936',
  '20060202103759',
  '20060205154436',
  '20060402185818',
  '20060610123125',
  '20070310185447',
  '20080704120852',
  '20111106082351',
  '20141120041602',
  '20150211074919',
  '20150813101949',
  '20150925044216',
  '20160207135730',
  '20160224063209',
  '20160320113947',
  '20160323010411',
  '20160328025927',
  '20160413115831',
  '20160501061622',
  '20160522152355',
 